# Machine Learning

Now, onto the fun stuff! Let's get our DataFrames again:

In [1]:
# import things
import pickle
import pandas as pd

## Posts and responses all together, with known genders
This dataframe will be used to predict gender, regardless of whether they were the poster or responder.

In [11]:
all_posts_responses = pd.read_pickle("all_posts_responses.pkl")
all_posts_responses.head()

,gender,text,tokens,text_length,avg_slen,hedges,source
0,M,"""I agree 100% with President Trump's decision....","[``, I, agree, 100, %, with, President, Trump,...",57,14.25,0,fb_congress_post
1,M,It's time to put Obama in the history books. W...,"[It, 's, time, to, put, Obama, in, the, histor...",37,18.50,0,fb_congress_post
2,M,Shameful.,"[Shameful, .]",2,2.00,0,fb_congress_post
3,M,"In Texas, we're proud to say Merry Christmas.","[In, Texas, ,, we, 're, proud, to, say, Merry,...",11,11.00,0,fb_congress_post
4,M,"In Texas, we're proud to say Merry Christmas.","[In, Texas, ,, we, 're, proud, to, say, Merry,...",11,11.00,0,fb_congress_post


In [14]:
# gender distribution
all_posts_responses.gender.value_counts()

M    148966
W    105707
Name: gender, dtype: int64

In [13]:
# group by gender
all_posts_responses.groupby('gender')['text_length', 'avg_slen', 'hedges'].mean()

,text_length,avg_slen,hedges
gender,,,
M,36.343998,12.686895,0.052938
W,35.803561,12.274098,0.039487


All of these stats are very similar, regardless of gender (even the hedges, which I predicted would be more prominent in female posters). This makes me believe that simply looking at text would be our best bet here.

## Responses to posts, genders visible
This dataframe will be used to predict gender of poster and responder, based on responder's reaction.

In [2]:
gender_visible_df = pd.read_pickle("gender_visible_df.pkl")
gender_visible_df.head()

,op_id,op_gender,responder_id,responder_gender,post_text,response_text,source,post_tokens,response_tokens,post_length,response_length,post_avg_slen,response_avg_slen,post_hedges,response_hedges
0,102,W,1196122,M,Thanks for the follow! I followed back :) I ...,One day at a time! =],fitocracy,"[Thanks, for, the, follow, !, I, followed, bac...","[One, day, at, a, time, !, =, ]]",25,8,12.5,4.000000,0,0
1,104,W,5867,W,I've decided I have this crazy goal of running...,Crazy is synonymous with awesome in this case.,fitocracy,"[I, 've, decided, I, have, this, crazy, goal, ...","[Crazy, is, synonymous, with, awesome, in, thi...",13,9,13.0,9.000000,0,0
2,104,W,1635,M,"Umm, yeah, so those are assisted pull-ups. Bef...",You can enter assisted pullups. Select pullups...,fitocracy,"[Umm, ,, yeah, ,, so, those, are, assisted, pu...","[You, can, enter, assisted, pullups, ., Select...",19,57,9.5,11.400000,0,0
3,117,M,8520,W,dam gurl lookin mad tone in dat pp holla bb,"Mirin 3% bodyfat? Yeah, you are.",fitocracy,"[dam, gurl, lookin, mad, tone, in, dat, pp, ho...","[Mirin, 3, %, bodyfat, ?, Yeah, ,, you, are, .]",10,10,10.0,5.000000,0,0
4,117,M,29126,M,What's up there bear mode?,"Hey! I just started a new job, so things are s...",fitocracy,"[What, 's, up, there, bear, mode, ?]","[Hey, !, I, just, started, a, new, job, ,, so,...",7,26,7.0,8.666667,0,0


In [3]:
gender_visible_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396766 entries, 0 to 396765
Data columns (total 15 columns):
op_id                396766 non-null object
op_gender            396766 non-null object
responder_id         396766 non-null object
responder_gender     396766 non-null object
post_text            396766 non-null object
response_text        396766 non-null object
source               396766 non-null object
post_tokens          396766 non-null object
response_tokens      396766 non-null object
post_length          396766 non-null int64
response_length      396766 non-null int64
post_avg_slen        396766 non-null float64
response_avg_slen    396766 non-null float64
post_hedges          396766 non-null int64
response_hedges      396766 non-null int64
dtypes: float64(2), int64(4), object(9)
memory usage: 45.4+ MB


Everything looks great! Let's recap some basic stats:

In [4]:
# poster gender distribution
gender_visible_df.op_gender.value_counts()

M    237339
W    159427
Name: op_gender, dtype: int64

In [5]:
# responder gender distribution
gender_visible_df.responder_gender.value_counts()

M    217639
W    179127
Name: responder_gender, dtype: int64

In [6]:
gender_visible_df.groupby(['source','op_gender','responder_gender'])['post_length','response_length',
                                                                     'post_avg_slen','response_avg_slen'].mean()

post_length  response_length  \
source    op_gender responder_gender                                 
fitocracy M         M                   19.023550        16.612086   
                    W                   15.503597        31.527746   
          W         M                   19.160420        15.814585   
                    W                   18.134494        40.720844   
reddit    M         M                   48.399651        38.106992   
                    W                   62.564599        48.993869   
          W         M                   55.883847        50.545774   
                    W                   58.729640        47.042140   

                                      post_avg_slen  response_avg_slen  
source    op_gender responder_gender                                    
fitocracy M         M                      9.677136           8.384321  
                    W                      8.910398          11.419609  
          W         M                      9.066472           7.888464  
                    W                      9.586728          13.561374  
reddit    M         M                     13.993324          12.982770  
                    W                     15.221700          13.782006  
          W         M                     14.828193          14.161640  
                    W                     15.947841          14.290182

So Reddit posts seem to be longer overall than posts on Fitocracy. This makes me believe that this may not be such a good statistic to use for machine learning. The more important factor may be the actual content of the text.

In [8]:
# hedges

# male posters with hedges
male_hedges_post = gender_visible_df[(gender_visible_df.op_gender == 'M') & (gender_visible_df.post_hedges != 0)]
# male responders with hedges
male_hedges_response = gender_visible_df[(gender_visible_df.responder_gender == 'M') & (gender_visible_df.response_hedges != 0)]

# female posters with hedges
female_hedges_post = gender_visible_df[(gender_visible_df.op_gender == 'W') & (gender_visible_df.post_hedges != 0)]
# female responders with hedges
female_hedges_response = gender_visible_df[(gender_visible_df.responder_gender == 'W') & (gender_visible_df.response_hedges != 0)]

print('percentage male posters with hedges:',
      len(male_hedges_post.index) / gender_visible_df.op_gender.value_counts()['M'])

print('percentage female posters with hedges:',
      len(female_hedges_post.index) / gender_visible_df.op_gender.value_counts()['W'])

print('percentage male responders with hedges:',
      len(male_hedges_response.index) / gender_visible_df.responder_gender.value_counts()['M'])

print('percentage female responders with hedges:',
      len(female_hedges_response.index) / gender_visible_df.responder_gender.value_counts()['W'])

percentage male posters with hedges: 0.05012239876295088
percentage female posters with hedges: 0.03971723735628219
percentage male responders with hedges: 0.053712799636094634
percentage female responders with hedges: 0.036292686194711016


This is actually the opposite trend that I wanted to see. It was my belief coming in that female posters/responders would use more hedges, but that is in fact not the case. But Reddit was actually skewed in the direction I wanted! What does just Reddit data look like?

In [10]:
reddit = gender_visible_df[gender_visible_df.source == 'reddit']

# male posters with hedges
male_hedges_post = reddit[(reddit.op_gender == 'M') & (reddit.post_hedges != 0)]
# male responders with hedges
male_hedges_response = reddit[(reddit.responder_gender == 'M') & (reddit.response_hedges != 0)]

# female posters with hedges
female_hedges_post = reddit[(reddit.op_gender == 'W') & (reddit.post_hedges != 0)]
# female responders with hedges
female_hedges_response = reddit[(reddit.responder_gender == 'W') & (reddit.response_hedges != 0)]

print('percentage male posters with hedges:',
      len(male_hedges_post.index) / reddit.op_gender.value_counts()['M'])

print('percentage female posters with hedges:',
      len(female_hedges_post.index) / reddit.op_gender.value_counts()['W'])

print('percentage male responders with hedges:',
      len(male_hedges_response.index) / reddit.responder_gender.value_counts()['M'])

print('percentage female responders with hedges:',
      len(female_hedges_response.index) / reddit.responder_gender.value_counts()['W'])

percentage male posters with hedges: 0.11949497649408028
percentage female posters with hedges: 0.16261107313738893
percentage male responders with hedges: 0.1084285233662827
percentage female responders with hedges: 0.15475588160890463


So this looks how I'd expect. It seems to be the Fitocracy data that is messing with my hypothesis. Thus, using the hedge data in machine learning will not work for this combined data set. Maybe we should just focus on the text.

## Solidifying ML plan

So what do I actually want to be able to predict?

1. Simply the gender of the poster/responder, based on the text alone (ignoring whether they are the poster or responder).
2. A more interesting and complicated topic would be analyzing the responder's text and predicting both their own gender and the original poster's gender. This way, we could see if different genders really do respond differently given who they are responding to. Let's make a dataframe that would fit this purpose:

In [15]:
# getting just the columns we want
responder_df = gender_visible_df[['op_gender','responder_gender','response_text',
                                  'response_tokens','response_length','response_avg_slen']]

responder_df.head()

,op_gender,responder_gender,response_text,response_tokens,response_length,response_avg_slen
0,W,M,One day at a time! =],"[One, day, at, a, time, !, =, ]]",8,4.000000
1,W,W,Crazy is synonymous with awesome in this case.,"[Crazy, is, synonymous, with, awesome, in, thi...",9,9.000000
2,W,M,You can enter assisted pullups. Select pullups...,"[You, can, enter, assisted, pullups, ., Select...",57,11.400000
3,M,W,"Mirin 3% bodyfat? Yeah, you are.","[Mirin, 3, %, bodyfat, ?, Yeah, ,, you, are, .]",10,5.000000
4,M,M,"Hey! I just started a new job, so things are s...","[Hey, !, I, just, started, a, new, job, ,, so,...",26,8.666667


In [16]:
# let's combine op_gender and responder_gender into one column, because this is what we want to predict
# original poster's gender is first character, responder's gender is second character
responder_df['gender_info'] = responder_df.op_gender + responder_df.responder_gender
responder_df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,op_gender,responder_gender,response_text,response_tokens,response_length,response_avg_slen,gender_info
0,W,M,One day at a time! =],"[One, day, at, a, time, !, =, ]]",8,4.000000,WM
1,W,W,Crazy is synonymous with awesome in this case.,"[Crazy, is, synonymous, with, awesome, in, thi...",9,9.000000,WW
2,W,M,You can enter assisted pullups. Select pullups...,"[You, can, enter, assisted, pullups, ., Select...",57,11.400000,WM
3,M,W,"Mirin 3% bodyfat? Yeah, you are.","[Mirin, 3, %, bodyfat, ?, Yeah, ,, you, are, .]",10,5.000000,MW
4,M,M,"Hey! I just started a new job, so things are s...","[Hey, !, I, just, started, a, new, job, ,, so,...",26,8.666667,MM


In [17]:
# drop the columns
responder_df.drop(labels=['op_gender','responder_gender'], axis=1, inplace=True)
responder_df.head()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,response_text,response_tokens,response_length,response_avg_slen,gender_info
0,One day at a time! =],"[One, day, at, a, time, !, =, ]]",8,4.000000,WM
1,Crazy is synonymous with awesome in this case.,"[Crazy, is, synonymous, with, awesome, in, thi...",9,9.000000,WW
2,You can enter assisted pullups. Select pullups...,"[You, can, enter, assisted, pullups, ., Select...",57,11.400000,WM
3,"Mirin 3% bodyfat? Yeah, you are.","[Mirin, 3, %, bodyfat, ?, Yeah, ,, you, are, .]",10,5.000000,MW
4,"Hey! I just started a new job, so things are s...","[Hey, !, I, just, started, a, new, job, ,, so,...",26,8.666667,MM


## Predict gender of poster/responder
Let's start simple! Given our `all_posts_responses` dataframe, can we predict the gender of the poster/responder?